In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy as db
from sqlalchemy import create_engine

In [2]:
path = 'clean_songs.csv'
db_path = 'sqlite:///spotify.db'

In [3]:
df = pd.read_csv(path)
df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
df.index.rename('id', inplace=True)
df = pd.DataFrame(df)
df.head()

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
id,,,,,,,,,,,,,,,,,,,,,
0,Downtown Life,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],1,1,False,0.660,0.892,...,0.0419,0.0971,0.000021,0.3500,0.767,110.089,268467,4,1988,7/27/1988
1,Everything Your Heart Desires,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],2,1,False,0.706,0.861,...,0.0411,0.1570,0.000001,0.2560,0.887,99.992,300467,4,1988,7/27/1988
2,I'm In Pieces,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],3,1,False,0.381,0.824,...,0.0408,0.1190,0.000000,0.0826,0.805,87.064,290467,4,1988,7/27/1988
3,Missed Opportunity,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],4,1,False,0.765,0.815,...,0.0429,0.5230,0.000000,0.0821,0.733,106.063,287600,4,1988,7/27/1988
4,Talking All Night,Ooh Yeah!,4EEyCnj2fZu1eInXpLfZZW,['Daryl Hall & John Oates'],['77tT1kLj6mCWtFNqiOmP9H'],5,1,False,0.745,0.787,...,0.0451,0.2190,0.000110,0.0910,0.902,120.925,274400,4,1988,7/27/1988


In [7]:
engine = create_engine(db_path)
conn = engine.connect()

In [8]:
attributes_df = df[['explicit', 'danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]

In [9]:
attributes_df.index.rename('attribute_id', inplace=True)

attributes_df.to_sql(name= 'attributes', con = engine, if_exists='replace')

In [17]:
artist_df = df[["artists"]]
artist_df.index.rename("artist_id", inplace=True)
artist_df.to_sql(name = "artist", con = engine, if_exists='replace')

,artists
artist_id,
0,['Daryl Hall & John Oates']
1,['Daryl Hall & John Oates']
2,['Daryl Hall & John Oates']
3,['Daryl Hall & John Oates']
4,['Daryl Hall & John Oates']


In [11]:
# Album Table
album = df.groupby('album')
album = album['album_id'].unique()
album = album.reset_index()
album = album.drop('album_id', axis=1)
album = pd.DataFrame(album)
album.index.rename("album_id", inplace=True)
album.head()

,album
album_id,
0,"""C"" For Chicago"
1,"""Make the Yuletide Gay"" Original Motion Pictur..."
2,"""Territory"" Shall Mean the Universe"
3,"""There's Always a Way Out"""
4,"""Waking the Cobra"", Vocal Meditations on the C..."


In [14]:
# Song Table
song = df[['name', 'track_number', 'disc_number', 'duration_ms', 'time_signature', 'year', 'release_date']]

In [15]:
# add the album DataFrame to the SQL database
album.to_sql(name = 'album', con=engine, if_exists='replace')
song.to_sql(name = 'song', con=engine, if_exists='replace')